In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import math
import time

In [2]:
dataArima = pd.read_parquet("Data/agg_table_new_zones8Bot.parquet")
#The already binned and aggregated dataset


In [ ]:
#Preparing the data for the ARIMA and creating a pivot table to predict for all 139 zones
dataArima["time_bin"] = pd.to_datetime(dataArima["time_bin"])
pivot_data = dataArima.pivot_table(index='time_bin', columns='h3_index', values='order_count', fill_value=0)

In [ ]:
#As there were some time bins without a value and not in the table we have to fill it with zeros
start_time_full = pivot_data.index.min()
end_time_full = pivot_data.index.max()
full_index_all_zones = pd.date_range(start=start_time_full, end=end_time_full, freq='15min')
pivot_data = pivot_data.reindex(full_index_all_zones, fill_value=0)

In [ ]:
#We use fourier features to recreate seasonal patterns with the help of sin and cos waves
def create_fourier_features(index, seasonal_period, k):
    n_samples = len(index)
    time = np.arange(n_samples)
    features = {}
    for i in range(1, k + 1):
        features[f'sin_{i}'] = np.sin(2 * np.pi * i * time / seasonal_period)
        features[f'cos_{i}'] = np.cos(2 * np.pi * i * time / seasonal_period)
    return pd.DataFrame(features, index=index)

In [ ]:
seasonal_period = 96 #(24*4) One day 
k = 24 #High value of fourier features to learn spikes
pdq_order = (2, 0, 1) # (p, d, q) with d=0 (AR) (I) (MA)
all_fourier_features = create_fourier_features(pivot_data.index, seasonal_period, k) # Preparation of the exogene variables with the data set
split_point_idx = int(len(pivot_data) * 0.875) #We use only test and train split as we don't need to validate. The value is unobserved by the model
results = {}

# Each 48 the values are refitted
refit_interval = 48

# preparing loop
total_zones = len(pivot_data.columns)
start_time_total = time.time()

In [ ]:
#We are going zone by zone to determine the values
for i, zone_id in enumerate(pivot_data.columns):
    print(f"\n--- Bearbeite Zone {i+1}/{total_zones}: {zone_id} ---")
    zone_series = pivot_data[zone_id]
    zone_series_log = np.log1p(zone_series)

    # Train/Test Split for this zone
    train_endog_zone = zone_series_log.iloc[:split_point_idx]
    test_endog_zone = zone_series_log.iloc[split_point_idx:]
    test_actual_zone = zone_series.iloc[split_point_idx:]

    train_exog_zone = all_fourier_features.iloc[:split_point_idx]
    test_exog_zone = all_fourier_features.iloc[split_point_idx:]

    # We use a walk forward for the training by using the variables
    history_endog = train_endog_zone.copy() # endogene variables (log)
    history_exog = train_exog_zone.copy() # exogene variables
    predictions_log_zone = [] # List of log predictions for this zone
    current_model_fit = None # Platzhalter für das aktuell gültige Modell
    zone_start_time = time.time()

    for t in range(len(test_endog_zone)):
        if t % refit_interval == 0 or current_model_fit is None:
            # Train model with history data during refit interval
            model = SARIMAX(endog=history_endog,
                            exog=history_exog,
                            order=pdq_order,
                            seasonal_order=(0, 0, 0, 0))
            
            try:
                # We enforce stationarity, by letting it on standard true
                #Maxiter is the max number of iterations
                current_model_fit = model.fit(disp=False, maxiter=200, method='lbfgs') #lbfgs is our optimization model
            except Exception as e:
                print(f"Error refitting zone {zone_id} at step {t+1}: {e}")
                # Fallback: If refit fails
                if current_model_fit is None:
                     print(f"Termination because of this zone here: {zone_id}")
                     predictions_log_zone = [np.nan] * len(test_endog_zone)
                     break # Next zone

        # Predictions with actual model
        try:
            exog_next_step = test_exog_zone.iloc[[t]] # Prepare exogenes for next step
            yhat_log = current_model_fit.forecast(steps=1, exog=exog_next_step)  # Predict one step
            predictions_log_zone.append(yhat_log.iloc[0])  #Extract value
        except Exception as e:
             print(f"Error predicting {zone_id} at step {t+1}: {e}")
             if predictions_log_zone:
                 predictions_log_zone.append(predictions_log_zone[-1])
             else:
                 predictions_log_zone.append(np.nan)

        # Add observation to next step
        obs_log = test_endog_zone.iloc[[t]]
        history_endog = pd.concat([history_endog, obs_log]) # Endog extended
        history_exog = pd.concat([history_exog, test_exog_zone.iloc[[t]]]) # Exog extended

    zone_end_time = time.time()
    #Calculate metrics for each zone
    if len(predictions_log_zone) == len(test_actual_zone):
        predictions_zone = np.expm1(np.array(predictions_log_zone))
        predictions_zone[predictions_zone < 0] = 0

        mae = mean_absolute_error(test_actual_zone, predictions_zone)
        mse = mean_squared_error(test_actual_zone, predictions_zone)
        rmse = math.sqrt(mse)
        r2 = r2_score(test_actual_zone, predictions_zone)

        nonzero_indices = test_actual_zone != 0
        actual_vals = test_actual_zone.values
        forecast_vals = predictions_zone

        if np.any(nonzero_indices):#for zero division
            mape = np.mean(np.abs((actual_vals[nonzero_indices] - forecast_vals[nonzero_indices]) / actual_vals[nonzero_indices])) * 100
        else:
            mape = np.nan
    else: #If it breaks
        mae, mse, rmse, r2, mape = [np.nan] * 5
        predictions_zone = np.full(len(test_actual_zone), np.nan)


    results[zone_id] = {
        'predictions': predictions_zone,
        'actual': test_actual_zone.values,
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
        'mape': mape
    }
    print(f"Zone {zone_id} - RMSE: {rmse:.2f}, R2: {r2:.4f}")


total_end_time = time.time()


--- Bearbeite Zone 1/139: 88329b430dfffff ---
Zone 88329b430dfffff - Refitting bei Schritt 1...
Zone 88329b430dfffff - Refitting bei Schritt 49...
Zone 88329b430dfffff abgeschlossen in 6.00 Sekunden.
Zone 88329b430dfffff - RMSE: 0.24, R2: -0.0070

--- Bearbeite Zone 2/139: 88329b4325fffff ---
Zone 88329b4325fffff - Refitting bei Schritt 1...


c:\Users\freud\anaconda3\envs\tf310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\freud\anaconda3\envs\tf310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


Zone 88329b4325fffff - Refitting bei Schritt 49...


c:\Users\freud\anaconda3\envs\tf310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\freud\anaconda3\envs\tf310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


Zone 88329b4325fffff abgeschlossen in 6.07 Sekunden.
Zone 88329b4325fffff - RMSE: 0.01, R2: 0.0000

--- Bearbeite Zone 3/139: 88329b432dfffff ---
Zone 88329b432dfffff - Refitting bei Schritt 1...


...

Zone 88329b58e3fffff - Refitting bei Schritt 49...


c:\Users\freud\anaconda3\envs\tf310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


Zone 88329b58e3fffff abgeschlossen in 9.92 Sekunden.
Zone 88329b58e3fffff - RMSE: 4.68, R2: 0.8862

--- Bearbeite Zone 85/139: 88329b58e5fffff ---
Zone 88329b58e5fffff - Refitting bei Schritt 1...
Zone 88329b58e5fffff - Refitting bei Schritt 49...
Zone 88329b58e5fffff abgeschlossen in 6.99 Sekunden.
Zone 88329b58e5fffff - RMSE: 1.79, R2: 0.5675

--- Bearbeite Zone 86/139: 88329b58e7fffff ---
Zone 88329b58e7fffff - Refitting bei Schritt 1...
Zone 88329b58e7fffff - Refitting bei Schritt 49...
Zone 88329b58e7fffff abgeschlossen in 8.03 Sekunden.
Zone 88329b58e7fffff - RMSE: 0.83, R2: 0.6769

--- Bearbeite Zone 87/139: 88329b58e9fffff ---
Zone 88329b58e9fffff - Refitting bei Schritt 1...


c:\Users\freud\anaconda3\envs\tf310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


Zone 88329b58e9fffff - Refitting bei Schritt 49...
Zone 88329b58e9fffff abgeschlossen in 7.17 Sekunden.
Zone 88329b58e9fffff - RMSE: 4.64, R2: 0.7071

--- Bearbeite Zone 88/139: 88329b58ebfffff ---
Zone 88329b58ebfffff - Refitting bei Schritt 1...
Zone 88329b58ebfffff - Refitting bei Schritt 49...


In [ ]:
#Calculate NRMSE
for zone_id, res in results.items():
    if 'rmse' in res and 'actual' in res and not np.isnan(res['rmse']):
        actual_vals = res['actual']
        rmse = res['rmse']

        mean_actual = np.mean(actual_vals)
        nrmse = (rmse / mean_actual) * 100 if mean_actual != 0 else np.nan 
        results[zone_id]['nrmse'] = nrmse
    else:
        results[zone_id]['nrmse'] = np.nan
#Calculate the averages of all 139 zones
avg_rmse = np.nanmean([res['rmse'] for res in results.values()])
avg_r2 = np.nanmean([res['r2'] for res in results.values()])
avg_mape = np.nanmean([res['mape'] for res in results.values()])
avg_mae = np.nanmean([res['mae'] for res in results.values()]) # nanmean because of nans

print(f"Average. RMSE: {avg_rmse:.2f}")
print(f"Average. MAPE: {avg_mape:.2f}%")
print(f"Average. MAE: {avg_mae:.2f}%")
print(f"Average. R2: {avg_r2:.4f}")

In [ ]:
#For our target zone for comparison
target_zone_id = '88329b5aa1fffff'

if target_zone_id in results:
    plt.figure(figsize=(15, 7))
    #Train data
    plt.plot(pivot_data.index[:split_point_idx],
             pivot_data[target_zone_id].iloc[:split_point_idx],
             label=f"Train (Zone {target_zone_id})")
    #Test data
    plt.plot(pivot_data.index[split_point_idx:],
             results[target_zone_id]['actual'],
             label=f"Test (Zone {target_zone_id})", color='orange')
    #Predictions
    plt.plot(pivot_data.index[split_point_idx:],
             results[target_zone_id]['predictions'],
             label=f"Walk-Forward Forecast (k={k}, d=0)", color='green', linestyle='--')
    plt.title(f"Walk-Forward Validation for zone {target_zone_id}")
    plt.legend()
    plt.show()

    # Export as numpy array for comparison
    predictions_to_export = results[target_zone_id]['predictions']
    export_filename = 'prognose_arima.npy' # Fixed filename
    np.save(export_filename, predictions_to_export)

else:
    print(f"Zone not found: This is a list of valid zones:  {list(results.keys())}")